In [6]:
%pip install kronoslabs

from kronoslabs import KronosLabs

def generate_badminton_feedback(analysis_output, api_key):
    """
    Generate personalized badminton feedback using LLM
    
    Args:
        analysis_output (str): The raw analysis output from your comparison method
        api_key (str): KronosLabs API key
    
    Returns:
        dict: Structured feedback with score, injury_risk, and tips
    """
    
    client = KronosLabs(api_key=api_key)
    
    prompt = f"""You are an expert badminton coach and sports physiotherapist specializing in injury prevention. 

Analyze this badminton smash form comparison data and provide detailed feedback:

{analysis_output}

Based on this analysis, provide:

1. OVERALL PERFORMANCE SCORE (0-100):
   - Consider technique accuracy, injury risk, and power generation
   - Deduct points for form issues that increase injury risk
   - Award points for good mechanics

2. INJURY RISK ASSESSMENT:
   - Rate as: LOW, MODERATE, or HIGH
   - Focus especially on shoulder injury risk (most common in badminton)
   - Consider joint angles, extension, and body mechanics

3. SPECIFIC CORRECTIVE TIPS:
   - For each major issue identified, provide:
     * What's wrong and why it matters
     * How to fix it with specific drills/exercises
     * Why this correction prevents injury
   - Prioritize issues by injury risk (shoulder issues first)

4. POSITIVE REINFORCEMENT:
   - Note what they're doing well
   - Encourage continued practice

Format your response as JSON:
{{
  "overall_score": <number 0-100>,
  "injury_risk": "<LOW|MODERATE|HIGH>",
  "injury_risk_explanation": "<brief explanation of main injury concerns>",
  "critical_issues": [
    {{
      "body_part": "<shoulder/elbow/wrist/hip>",
      "problem": "<what's wrong>",
      "injury_risk": "<why this causes injury>",
      "correction": "<specific fix>",
      "drill": "<practice drill to improve>"
    }}
  ],
  "positive_feedback": "<what they're doing well>",
  "summary": "<2-3 sentence overall assessment>"
}}

Be encouraging but honest. Focus on injury prevention and safe technique improvement."""

    try:
        response = client.chat.completions.create(
            prompt=prompt,
            model="hermes",
            temperature=0.7,
            is_stream=False
        )
        
        feedback_text = response.choices[0].message.content
        
        # Try to parse JSON response
        import json
        # Extract JSON from response (in case there's additional text)
        start_idx = feedback_text.find('{')
        end_idx = feedback_text.rfind('}') + 1
        
        if start_idx != -1 and end_idx != 0:
            json_str = feedback_text[start_idx:end_idx]
            feedback_data = json.loads(json_str)
        else:
            # Fallback if JSON parsing fails
            feedback_data = {
                "overall_score": 70,
                "injury_risk": "MODERATE",
                "injury_risk_explanation": "Analysis completed but formatting failed",
                "critical_issues": [],
                "positive_feedback": "Keep practicing!",
                "summary": feedback_text
            }
        
        return feedback_data
        
    except Exception as e:
        print(f"Error generating feedback: {e}")
        return {
            "overall_score": 0,
            "injury_risk": "UNKNOWN",
            "injury_risk_explanation": f"Error: {str(e)}",
            "critical_issues": [],
            "positive_feedback": "",
            "summary": "Unable to generate feedback. Please try again."
        }


# Example usage
if __name__ == "__main__":
    # Your analysis output
    analysis_text = """
IMPACT POINT ANALYSIS
============================================================
User impact at frame: 129.0
Reference impact at frame: 49.0
============================================================
BADMINTON SMASH FORM ANALYSIS
============================================================
📐 ELBOW ANGLE AT IMPACT:
   Your angle: 159.1°
   Reference: 174.9°
   Difference: -15.8°
   ⚠️  Your elbow is too bent. Extend more at contact for maximum reach
       and racket head speed.
💪 SHOULDER ANGLE AT IMPACT:
   Your angle: 81.3°
   Reference: 23.1°
   Difference: +58.2°
   ⚠️  Your shoulder is over-rotated. This may cause loss of control.
🎯 CONTACT POINT HEIGHT:
   Your height: -0.100
   Reference: -0.149
   Difference: -0.058
   ⚠️  Reach higher! Full arm extension maximizes racket head speed.
   💪 Keep practicing! Focus on the key areas mentioned above.
"""
    
    # Generate feedback
    api_key = "kl_8de5d1021ab88d725db4426791bcd805cf189a467e0465488a0567c0647b64bb"
    feedback = generate_badminton_feedback(analysis_text, api_key)
    
    print("=== FEEDBACK GENERATED ===")
    print(f"Overall Score: {feedback['overall_score']}/100")
    print(f"Injury Risk: {feedback['injury_risk']}")
    print(f"\nRisk Explanation: {feedback['injury_risk_explanation']}")
    print(f"\nSummary: {feedback['summary']}")
    print(f"\nPositive Feedback: {feedback['positive_feedback']}")
    
    print("\n=== CRITICAL ISSUES ===")
    for i, issue in enumerate(feedback['critical_issues'], 1):
        print(f"\n{i}. {issue['body_part'].upper()}")
        print(f"   Problem: {issue['problem']}")
        print(f"   Injury Risk: {issue['injury_risk']}")
        print(f"   Correction: {issue['correction']}")
        print(f"   Drill: {issue['drill']}")

Note: you may need to restart the kernel to use updated packages.
=== FEEDBACK GENERATED ===
Overall Score: 65/100
Injury Risk: MODERATE

Risk Explanation: Significant shoulder over-rotation and insufficient elbow extension increase risk of rotator cuff strains and elbow tendinitis. Contact point too low reduces power efficiency.

Summary: Technique shows good foundation with strong racket speed potential, but requires correction of shoulder over-rotation, insufficient elbow extension, and contact point height to reduce injury risk and optimize power. Focus on maintaining stable shoulder position while rotating torso, and practicing full arm extension with proper contact height.

Positive Feedback: Excellent racket head speed generation potential and good overall smash mechanics. Your torso rotation and lower body engagement appear strong. Keep focusing on these positive elements while correcting the specific form issues.

=== CRITICAL ISSUES ===

1. SHOULDER
   Problem: Over-rotation 